In [1]:
# pip3 install torch torchvision
# pip3 install transformers sentencepiece
# https://huggingface.co/Jean-Baptiste/camembert-ner
# https://huggingface.co/docs/transformers/quicktour
import random
import pandas as pd
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification

## load online model and save to local with custom name
'''
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner", use_fast=False)
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")
model.save_pretrained("model-camembert-ner")
tokenizer.save_pretrained("tokenizer-camembert-ner")
'''

tokenizer = AutoTokenizer.from_pretrained("tokenizer-camembert-ner", use_fast=False)
model = AutoModelForTokenClassification.from_pretrained("model-camembert-ner")

nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")

In [2]:
path = "data/data_set_final.csv"
df = pd.read_csv(path)

In [3]:
def get_entity(data):
    n = len(data)
    words = [ [] for _ in range(n)]
    entities = [ [] for _ in range(n)]

    for i in range(n):
        for r in nlp(data[i]):
            words[i].append(r['word'])
            entities[i].append(r['entity_group'])
    return words, entities

In [4]:
def most_frequent(l):
    if l: return max(l, key = l.count)
    return 'MISC'

In [5]:
def get(column):
    res = pd.DataFrame()
    res[column] = random.sample(df[column].tolist(), 2000)
    words, entities = get_entity(res[column])
    res["words"] = words
    res["entities"] = entities
    res["entity"] = [most_frequent(e) for e in res["entities"]]
    return res

In [6]:
%%time
# about 1min per 1000 lignes
adresse = get("adresse")
adresse["entity"].value_counts()

CPU times: user 1min 55s, sys: 2.47 s, total: 1min 58s
Wall time: 2min 8s


LOC     1878
PER       96
MISC      25
ORG        1
Name: entity, dtype: int64

In [7]:
%%time
nom = get("nom")
nom["entity"].value_counts()

CPU times: user 1min 44s, sys: 2.35 s, total: 1min 46s
Wall time: 1min 54s


PER     1590
MISC     195
LOC      188
ORG       27
Name: entity, dtype: int64

In [8]:
%%time
prenom = get("prenom")
prenom["entity"].value_counts()

CPU times: user 1min 38s, sys: 1.75 s, total: 1min 40s
Wall time: 1min 44s


PER     1588
MISC     230
LOC      156
ORG       26
Name: entity, dtype: int64